## 🛠️ **ETL (Extract, Transform, Load)**



#### **📂Procesamiento del 1er archivo: `steam_games.json.gz`**

- 🗑️ Eliminamos columnas que no vamos a usar: publisher, title, url, early_access, reviews_url, specs.

🔄 `ID:`
- Eliminamos nulos y repetidos

`release_date`:

- Extraigo el año y cambio el nombre de la columna a 'release_year'
- Reemplazamos los datos sin fecha por
- Cambiamos tipo de dato a int.

`price`:

- Asignmos valores correspondientes a precios con textos adicional.
- Rellenamos missing values buscando en tags si el juego es gratis.
- Rellena los valores nulos resultantes con -1.
- Convierte la columna 'price' a tipo float.

`genres:`

- Rellenamos la informacion faltante de genres, con los genres disponibles en la columna tags.

🔄 `app_name` en string

🔄 `tags` en string

🗃️ `Genres `: Rellenamos la informacion faltante de genres en cada juego, con los genres disponibles en la columna tag

#### 📦 **EXTRACT**

####  **Importamos las librerías que vamos a usar**


In [18]:
import pandas as pd  # Pandas se utiliza para el manejo y análisis de datos tabulares
import pyarrow as pa  # PyArrow se utiliza para trabajar con formatos de datos columnares y eficientes como Parquet
import pyarrow.parquet as pq  # Importamos Parquet
import re
import inspect

import ast  # AST (Abstract Syntax Trees) se utiliza para interpretar expresiones Python
import gzip
import json  # JSON se utiliza para trabajar con datos en formato JSON
import os  # OS proporciona funciones para interactuar con el sistema operativo
import time
import warnings  # Warnings se utiliza para gestionar advertencias y filtrarlas si es necesario
warnings.filterwarnings("ignore")
import data_utils
from data_utils import data_type_check, filtrar_valores_letras, extraer_anio_release
# Autoreload se utiliza para recargar automáticamente los módulos al realizar cambios
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### **Descomprimimos el archivos gz** 

In [19]:
# Obtenemos el tiempo de inicio de todo este ipynb 
start_time = time.time()

def descomprimir_archivos_gz(archivos_gz, carpeta_destino):
    for archivo_gz in archivos_gz:
        with gzip.open(archivo_gz, 'rb') as f_in:
            contenido = f_in.read()
            archivo_destino = os.path.join(carpeta_destino, os.path.splitext(os.path.basename(archivo_gz))[0])
            with open(archivo_destino, 'wb') as f_out:
                f_out.write(contenido)
        print(f'Archivo descomprimido: {archivo_destino}')

# Ejemplo de uso con una lista de archivos gz
archivo_gz_a_descomprimir = ['../0 Dataset/steam_games.json.gz']
carpeta_destino = '../0 Dataset/'

descomprimir_archivos_gz(archivo_gz_a_descomprimir, carpeta_destino)


Archivo descomprimido: ../0 Dataset/steam_games.json


## **📂Procesamiento del 1er archivo: `steam_games.json.gz`**


- Tomamos los datos del archivo JSON, los transformamos en un DataFrame y realizamos una primera observación de su contenido.

In [20]:
#Creamos una lista vacía llamada "rows" donde almacenaremos los datos del archivo JSON.

row = []
with open("../0 Dataset/steam_games.json", "r", encoding="utf-8") as archivo:
    for linea in archivo:
        try:
            objeto_json = json.loads(linea)
            row.append(objeto_json)
        except json.JSONDecodeError:
            print(f"Error de formato JSON en la línea: {linea}")

#Convertir la lista de objetos JSON en un DataFrame
df_steam_games = pd.DataFrame(row)
#Hacemos una copia del dataset sucio para explorarlo en el EDA
df_steam_games_sucio = pd.DataFrame(row)
# Veamos unos registros al azar
df_steam_games.head(2)


,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Usando la funcion personalizada `data_type_check` invocada desde `data_utils.py` podemos observar:
- Variables categóricas
- Variables numéricas
- Dimensiones del dataframe
- Nulos
- Tipos de datos
- Informacion acerca de los datos faltantes o nulos de cada columna    


In [21]:
data_type_check(df_steam_games)


 Resumen del dataframe 'df_steam_games': 

Dimensiones:  (120445, 13)
         columna  %_no_nulos  %_nulos  total_nulos tipo_dato
0      publisher       20.00    80.00        96362    object
1         genres       23.95    76.05        91593    object
2       app_name       26.68    73.32        88312    object
3          title       24.98    75.02        90360    object
4            url       26.68    73.32        88310    object
5   release_date       24.96    75.04        90377    object
6           tags       26.54    73.46        88473    object
7    reviews_url       26.68    73.32        88312    object
8          specs       26.12    73.88        88980    object
9          price       25.54    74.46        89687    object
10  early_access       26.68    73.32        88310    object
11            id       26.68    73.32        88312    object
12     developer       23.94    76.06        91609    object


Observamos que los tipos de datos no son los que necesitamos, vamos a transformar estos datos en los formatos correctos para su posterior uso.

#### 🔁 **TRANSFORM**
  * ✅ Columnas irrelevantes (que no nos sirven para el problema que queremos resolver)
   developer, id. 
  * ✅ Datos faltantes o nulos: **.dropna**
  * ✅ Registros o filas repetidas: **drop_duplicates**
  * ✅ Variables categoricas: Errores tipográficos

#### **Columnas "publisher","title","url","early_access","reviews_url","specs"**

Eliminamos estas columnas ya que son irrelevantes para el problema que queremos resolver:

In [22]:
print("Dimensiones previas a limpieza:")
print(df_steam_games.shape)

# Eliminamos columnas que no vamos a usar
df_steam_games.drop(columns=["publisher","title","url","early_access","reviews_url","specs"], inplace=True)

print("Dimensiones post limpieza:")
print(df_steam_games.shape)

Dimensiones previas a limpieza:
(120445, 13)
Dimensiones post limpieza:
(120445, 7)


**Transformamos en lote las columnas que no necesitan tratamiento especial**

In [23]:
#  Transformacion app_name, tags, developer a string.

df_steam_games = df_steam_games.astype({
    'app_name': 'string',
    'tags': 'string', 
    'developer': 'string',
    'id': 'string'
})

#### **ID**
Eliminamos repetidos y nulos

In [24]:
# Eliminar duplicados en la columna 'id'
df_steam_games.drop_duplicates(subset='id', inplace=True)
# Eliminar filas con valores nulos en la columna 'id'
df_steam_games.dropna(subset=['id'], inplace=True)
#Terminamos reseteando los indices
df_steam_games.reset_index(drop=True, inplace=True)
# Cambiamos el dato a user_id
df_steam_games.rename(columns={'id':'user_id'}, inplace=True)

#### **Developer**

In [25]:
# Reemplazamos los nulos con Dato Faltante
df_steam_games['developer'].fillna('Dato Faltante', inplace=True)
# Cambiar el tipo de dato a string
df_steam_games['developer'] = df_steam_games['developer'].astype('string')

In [26]:
data_type_check(df_steam_games)


 Resumen del dataframe 'df_steam_games': 

Dimensiones:  (32132, 7)
        columna  %_no_nulos  %_nulos  total_nulos       tipo_dato
0        genres       89.79    10.21         3282          object
1      app_name      100.00     0.00            1  string[python]
2  release_date       93.57     6.43         2066          object
3          tags       99.50     0.50          162  string[python]
4         price       95.71     4.29         1377          object
5       user_id      100.00     0.00            0  string[python]
6     developer      100.00     0.00            0  string[python]


#### **Transformación de 'release_date'**
- Cambiamos tipo de dato a str
- Se extrae el año de la fecha en la columna 'release_date' 
- Reemplazamos los datos sin fecha por 
- Llamamos a la funcion extraer_anio_release en data_utils.py  

In [27]:
# Aplica la función extraer_anio_release a la columna release_date

df_steam_games['release_year'] = df_steam_games['release_date'].apply(data_utils.extraer_anio_release)
# elimina la columna 'release_date'
df_steam_games = df_steam_games.drop('release_date', axis=1)
data_type_check(df_steam_games)


 Resumen del dataframe 'df_steam_games': 

Dimensiones:  (32132, 7)
        columna  %_no_nulos  %_nulos  total_nulos       tipo_dato
0        genres       89.79    10.21         3282          object
1      app_name      100.00     0.00            1  string[python]
2          tags       99.50     0.50          162  string[python]
3         price       95.71     4.29         1377          object
4       user_id      100.00     0.00            0  string[python]
5     developer      100.00     0.00            0  string[python]
6  release_year      100.00     0.00            0          object


#### **price**
Necesitamos trabajar con esta columna, pero encontramos valores de texto para promociones o indicando que el juego es gratis. 

- Asignamos valores correspondientes a precios con textos adicional.
- Convierte la columna 'price' a tipo float.
- Rellena los valores nulos resultantes con -1.

Visualizamos los valores no numericos en precios a solucionar usando una funcion invocada desde data_utils.py

In [28]:
precios_texto = filtrar_valores_letras(df_steam_games['price'].unique())
print(precios_texto)

['Free To Play', 'Free to Play', 'Free', 'Free Demo', 'Play for Free!', 'Install Now', 'Play WARMACHINE: Tactics Demo', 'Free Mod', 'Install Theme', 'Third-party', 'Play Now', 'Free HITMAN™ Holiday Pack', 'Play the Demo', 'Starting at $499.00', 'Starting at $449.00', 'Free to Try', 'Free Movie', 'Free to Use']


Filtramos los precios gratis a traves de buscar los datos faltantes en la columna tags

In [29]:
# Lista de palabras clave para buscar en las etiquetas
palabras_clave = ['Free to Play', 'Free', 'Free Demo', 'Play for Free!', 'Free Mod', 'Free to Try', 'Free Movie', 'Free to Use', 'Play the Demo' ]

print("Muestra de valores no numéricos antes de la limpieza:")
precio_ironbound = df_steam_games[df_steam_games['app_name'] == 'Ironbound']['price'].unique()
print(precio_ironbound)

# Verificar si el precio del juego es nulo
if df_steam_games['price'].isnull().any():
    # Verificar si alguna de las palabras clave está en las etiquetas para cada fila y si falta el precio
    df_steam_games['precio_faltante'] = df_steam_games['price'].isna() & df_steam_games['tags'].apply(lambda x: any(palabra in x for palabra in palabras_clave) if isinstance(x, list) else False)

    # Reemplazar los valores de precio faltantes con 0 si la fila correspondiente tiene alguna palabra clave en las etiquetas
    df_steam_games.loc[df_steam_games['precio_faltante'], 'price'] = 0

    # Eliminar la columna temporal
    df_steam_games.drop('precio_faltante', axis=1, inplace=True)

# Limpiar valores no numéricos
non_numeric_values = ['Free', 'Free To Play', 'Free Demo', 'Play for Free!', 'Free Mod', 'Install Now', 'Play Now', 'Third-party', 'Play WARMACHINE: Tactics Demo', 'Install Theme', 'Starting at', 'Free to Try', 'Free Movie', 'Free to Use', 'Play the Demo']
df_steam_games['price'] = df_steam_games['price'].apply(lambda x: 0 if x in non_numeric_values else x)

# Convertir la columna 'price' a tipo string para limpiar valores con texto adicional
df_steam_games['price'] = df_steam_games['price'].astype(str)

# Limpiar valores con texto adicional y convertir la columna 'price' a tipo float
df_steam_games['price'] = df_steam_games['price'].replace(to_replace=r'[^0-9.]', value='', regex=True)
df_steam_games['price'] = pd.to_numeric(df_steam_games['price'], errors='coerce')

# Verificar si hay valores nulos en la columna 'price'
if df_steam_games['price'].isnull().any():
    # Asignar -1 a los valores nulos en la columna 'price'
    df_steam_games['price'].fillna(-1, inplace=True)

# Convertir la columna 'price' a tipo int
df_steam_games['price'] = df_steam_games['price'].astype(int)


print("Muestra de valores no numéricos después de la limpieza:")
precio_ironbound = df_steam_games[df_steam_games['app_name'] == 'Ironbound']['price'].unique()
print(precio_ironbound)

data_type_check(df_steam_games)

Muestra de valores no numéricos antes de la limpieza:
['Free To Play']
Muestra de valores no numéricos después de la limpieza:
[0]

 Resumen del dataframe 'df_steam_games': 

Dimensiones:  (32132, 7)
        columna  %_no_nulos  %_nulos  total_nulos       tipo_dato
0        genres       89.79    10.21         3282          object
1      app_name      100.00     0.00            1  string[python]
2          tags       99.50     0.50          162  string[python]
3         price      100.00     0.00            0           int32
4       user_id      100.00     0.00            0  string[python]
5     developer      100.00     0.00            0  string[python]
6  release_year      100.00     0.00            0          object


#### **genres**:

- Rellenamos la informacion faltante de genre, con los genre disponibles en la columna tags
- Eliminamos generos innecesarios como Free to Play y Early Access
- Eliminamos la columna tags


In [30]:
print("Valores faltantes antes de el relleno de datos: ", df_steam_games['genres'].isnull().sum())

Valores faltantes antes de el relleno de datos:  3282


In [31]:
# Rellenamos la informacion faltante de genre, con los genre disponibles en la columna tags
print("Valores faltantes previos: ", df_steam_games['genres'].isnull().sum())

# Conjunto de géneros a copiar
generos_a_copiar = {'Simulation', 'Adventure', 'Strategy', 'Education', 'Photo Editing', 'Massively Multiplayer', 'Accounting', 'Video Production', 'Design & Illustration', 'Racing', 'Web Publishing', 'Utilities', 'Software Training', 'Sports', 'Action', 'Indie', 'Audio Production', 'Animation & Modeling', 'Casual', 'RPG'}

# Función para rellenar genres desde tags
def rellenar_genres(tags, genres):
    try:
        # Convertir las etiquetas (tags) a una lista
        tags_list = ast.literal_eval(tags)
        # Filtrar los géneros a copiar
        nuevos_generos = [genre for genre in generos_a_copiar if genre in tags_list]        
        # Eliminar 'Free to Play' y 'Early Access'
        no_deseados = ['Free to Play', 'Early Access']
        nuevos_generos = [genre for genre in nuevos_generos if genre not in no_deseados]        
        # Si no hay nuevos géneros, devolver los originales
        if not nuevos_generos:
            return genres        
        # Devolver los nuevos géneros como lista
        return nuevos_generos
    
    except (ValueError, SyntaxError):
        # Manejar errores de evaluación
        pass
    
    # Si hay algún error, o la evaluación no es una lista, devolver los géneros originales como lista
    return genres

# Aplicar la función para rellenar genres desde tags
df_steam_games['genres'] = df_steam_games.apply(lambda row: rellenar_genres(row['tags'], row['genres']), axis=1)


# Eliminar tags
df_steam_games = df_steam_games.drop('tags', axis=1)

# Observamos la cantidad de nulos
print("Valores faltantes luego de el relleno de datos: ", df_steam_games['genres'].isnull().sum())

Valores faltantes previos:  3282
Valores faltantes luego de el relleno de datos:  942


In [32]:
data_type_check(df_steam_games)


 Resumen del dataframe 'df_steam_games': 

Dimensiones:  (32132, 6)
        columna  %_no_nulos  %_nulos  total_nulos       tipo_dato
0        genres       97.07     2.93          942          object
1      app_name      100.00     0.00            1  string[python]
2         price      100.00     0.00            0           int32
3       user_id      100.00     0.00            0  string[python]
4     developer      100.00     0.00            0  string[python]
5  release_year      100.00     0.00            0          object


#### **📤 LOAD**

In [33]:
# Define la ruta del archivo Parquet
ruta_parquet = '../0 Dataset/steam_games_LISTO.parquet'
ruta_csv = '../0 Dataset/steam_games_LIMPIO.csv'
# Guarda los cambios en CSV
df_steam_games.to_csv(ruta_csv, index=False)

# Lee el archivo CSV en un DataFrame de pandas
steam_games = pd.read_csv(ruta_csv)

# Convierte el DataFrame de pandas a una tabla de PyArrow
table_steam_games = pa.Table.from_pandas(steam_games)

# Escribe la tabla en un archivo Parquet
pq.write_table(table_steam_games, ruta_parquet)

# Borra el archivo CSV ya que se prefiere Parquet
os.remove('../0 Dataset/steam_games_LIMPIO.csv')

# Deja información de muestra acerca de los archivos
data_type_check(steam_games)


 Resumen del dataframe 'steam_games': 

Dimensiones:  (32132, 6)
        columna  %_no_nulos  %_nulos  total_nulos tipo_dato
0        genres       97.07     2.93          942    object
1      app_name      100.00     0.00            1    object
2         price      100.00     0.00            0     int64
3       user_id      100.00     0.00            0     int64
4     developer      100.00     0.00            0    object
5  release_year      100.00     0.00            0     int64


In [34]:
df_steam_games

,genres,app_name,price,user_id,developer,release_year
0,"[Casual, Indie, Strategy, Action, Simulation]",Lost Summoner Kitty,4,761140,Kotoshiro,2018
1,"[Design & Illustration, Indie, Strategy, RPG]",Ironbound,0,643980,Secret Level SRL,2018
2,"[Casual, Sports, Indie, Simulation]",Real Pool 3D - Poolians,-1,670290,Poolians.com,2017
3,"[Adventure, Casual, Action]",弹炸人2222,0,767400,彼岸领域,2017
4,"[Casual, Sports, Indie, Action]",Log Challenge,2,773570,Dato Faltante,0000
...,...,...,...,...,...,...
32127,"[Casual, Indie, Strategy, Simulation]",Colony On Mars,1,773640,"Nikita ""Ghost_RUS""",2018
32128,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,4,733530,Sacada,2018
32129,"[Racing, Indie, Simulation]",Russian Roads,1,610660,Laush Dmitriy Sergeevich,2018
32130,"[Casual, Indie]",EXIT 2 - Directions,4,658870,"xropi,stev3ns",2017
